In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [1]:
renamed_columns = ['data','hora','precipitacao total,horario (mm)','pressao atmosferica ao nivel da estacao (mb)','pressao atmosferica max. na hora ant. (aut) (mb)','pressao atmosferica min. na hora ant. (aut) (mb)','radiation (kj/m2)','temperatura do ar - bulbo seco (°c)','temperatura do ponto de orvalho (°c)','temperatura maxima na hora ant. (aut) (°c)','temperatura minima na hora ant. (aut) (°c)','temperatura orvalho max. na hora ant. (aut) (°c)','temperatura orvalho min. na hora ant. (aut) (°c)','umidade rel. max. na hora ant. (aut) (%)','umidade rel. min. na hora ant. (aut) (%)','umidade relativa do ar, horaria (%)','vento direcao horaria (gr) (° (gr))','vento rajada maxima (m/s)','vento velocidade horaria (m/s)','region','state','station','station_code','latitude','longitude','height']
renamed_columns_en = ['date','hour','total precipitation (mm)','atmospheric pressure at station level (mb)','atmospheric pressure max. in the previous hour (mb)','atmospheric pressure min. in the previous hour (mb)','radiation (kj/m2)','air temperature - dry bulb (°c)','dew point temperature (°c)','max. temperature in the previous hour (°c)','min. temperature in the previous hour (°c)','dew temperature max. in the previous hour (°c)','dew temperature min. in the previous hour (°c)','relative humidity max. in the previous hour (%)','relative humidity min. in the previous hour (%)','air relative humidity (%)','wind direction (° (gr))','wind rajada maxima (m/s)','wind speed (m/s)','region','state','station','station_code','latitude','longitude','height']
abbreviation = ['date','hour','prcp', 'stp', 'smax', 'smin','gbrd','temp','dewp','tmax','tmin','dmax','dmin','hmax','hmin','hmdy','wdct', 'gust', 'wdsp', 'regi','prov','wsnm','inme','lat','lon','elvt']

def process_raw(df):
    """
    ! drop index
    ! rename the columns
    ! combine hour and date columns -> transform to date_time
    ! append columns data from all stations to keep the data granularity
    ! remove not useful columns
    ! return table 
    """
    df.drop(['index'],inplace=True, axis=1)
    df.columns = abbreviation
    df['date_time'] = pd.to_datetime(df['date'] + ' ' +  df['hour'])
    df.drop(['date','hour','regi','prov','wsnm','lat','lon','elvt'], inplace=True, axis=1)

    station_codes = list(df.station_code.unique())
    df_models = by_code(df,station_codes[0])
    if len(station_codes)>1:
        for i in station_codes[1:]:
            df_temp = by_code(df,i)
            df_models = pd.concat([df_models,df_temp],axis=1)
    return df_models

def by_code(df, station_code):
    """
    ! function that filters the dataframe by station_code and rename the columns by the status code
    """
    df_station = None
    df_station = df[df['station_code'] == station_code]
    df_station.index = df_station.iloc[:,-1]
    df_station.drop(['station_code','date_time','radiation (kj/m2)'],axis=1,inplace=True)
    df_station.columns = list(map(lambda x: station_code+'_'+x, df_station.columns))
    return df_station

def clean_na(df, na_value = -9999):
    """ 
    ! replace Na from dataset
    """
    df = df.replace(to_replace=na_value,value=np.NaN)
    df = df[df.iloc[:,1].first_valid_index():]
    df = df.fillna(method='ffill')
    return df

def make_dataset(stations, start_date, df):
    """
    ! filter raw data from date and stations code
    ! process data
    ! clean na
    """
    # df = df[df['date'] >= start_date]
    # df = df[df['station_code'].apply(lambda x: x in stations)]
    #df = process_raw(df)
    df = clean_na(df)
    return df.reset_index()


In [13]:
mydf = pd.read_csv("cleaned_south_east.csv")
#mydf = make_dataset(mydf['station_code'],"1991-10-14",mydf)
mydf.head()
# mydf.sample(3500).to_csv("cleaned_south_east.csv",sep=",",encoding="utf-8",na_rep='')

,index,date,hour,total precipitation (mm),pressao atmosferica ao nivel da estacao (mb),atmospheric pressure max. in the previous hour (mb),atmospheric pressure min. in the previous hour (mb),radiation (kj/m2),air temperature - dry bulb,dew point temperature,...,wind direction,wind rajada maxima (m/s),wind speed (m/s),region,state,station,station_code,latitude,longitude,height
0,647,2001-09-25,23:00,0.0,940.3,940.5,940.2,0,19.6,18.5,...,16,5.7,2.2,SE,SP,BAURU,A705,-22.358056,-49.028889,666
1,4046,2000-10-22,14:00,0.0,1009.5,1010.4,1009.5,1622,26.1,21.3,...,197,2.3,0.8,SE,RJ,ECOLOGIA AGRICOLA,A601,-22.800000,-43.683333,33
2,4469,2000-11-09,5:00,0.0,1008.7,1009.2,1008.7,0,20.7,17.6,...,343,2.8,0.6,SE,RJ,ECOLOGIA AGRICOLA,A601,-22.800000,-43.683333,33
3,5032,2000-12-02,16:00,0.0,1008.8,1009.6,1008.8,2436,26.2,20.6,...,165,5.1,2.3,SE,RJ,ECOLOGIA AGRICOLA,A601,-22.800000,-43.683333,33
4,5549,2000-12-24,5:00,0.0,1009.6,1010.5,1009.6,0,24.6,21.4,...,9,3.6,1.4,SE,RJ,ECOLOGIA AGRICOLA,A601,-22.800000,-43.683333,33


In [28]:
for x in mydf['relative humidity max. in the previous hour (%)'].unique():
    print(x)

94
86
82
73
83
95
91
52
51
39
56
80
98
79
78
63
43
67
57
41
85
46
93
48
61
30
96
29
62
54
92
64
60
87
50
81
58
55
77
89
75
49
66
44
88
76
84
70
72
90
97
68
65
71
42
69
59
53
27
47
74
37
38
26
31
45
40
36
34
33
0
24
35
28
32
25
